In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
df.shape

## Q1. Downloading the data

Read the data for January. How many columns are there?

In [ ]:
len(df.columns)

## Q2. Computing duration
What's the standard deviation of the trips duration in January?

In [ ]:
df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df["duration"] = df["duration"].dt.total_seconds() / 60
df["duration"].head()

In [ ]:
df["duration"].std()

## Q3. Dropping outliers
What fraction of the records left after you dropped the outliers?

In [ ]:
(len(df[(df.duration<=60) & (df.duration>=1)]) / df.shape[0]) * 100

In [ ]:
df = df[(df.duration<=60) & (df.duration>=1)].copy()

## Q4. One-hot encoding
What's the dimensionality of this matrix (number of columns)?

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
categoricals = ['PULocationID', 'DOLocationID']

In [ ]:
df[categoricals] = df[categoricals].astype(str)

In [ ]:
df_enc = df[categoricals].to_dict(orient="records")

In [ ]:
df_enc[:5]

In [ ]:
df[categoricals].head()

In [ ]:
vectorizer = DictVectorizer()

In [ ]:
X_train = vectorizer.fit_transform(df_enc)

In [ ]:
print(f"shape: {X_train.shape}")

## Q5. Training a model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [ ]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)

In [ ]:
print(f"Train rmse: {rmse}")

## Q6. Evaluating the model

In [ ]:
def remove_outliers(df):
    """
    Add duration and remove outliers
    """
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].dt.total_seconds() / 60

    return df[(df.duration<=60) & (df.duration>=1)].copy()    

In [ ]:
df_raw_eval = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
df_raw_eval = remove_outliers(df_raw_eval)

In [ ]:
val_dicts = df_raw_eval[categoricals].to_dict(orient='records')

In [ ]:
X_val = vectorizer.transform(val_dicts) 
y_val = df_raw_eval.duration.values

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
print(f'test rmse: {root_mean_squared_error(y_val, y_pred)}')